In [1]:
import pandas as pd
import numpy as np
from scipy import  stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller as ADF
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from dateutil.relativedelta import relativedelta

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [120]:
pd.date_range(start='2000-01-31', end='2020-01-31', freq='3M')

DatetimeIndex(['2000-01-31', '2000-04-30', '2000-07-31', '2000-10-31',
               '2001-01-31', '2001-04-30', '2001-07-31', '2001-10-31',
               '2002-01-31', '2002-04-30', '2002-07-31', '2002-10-31',
               '2003-01-31', '2003-04-30', '2003-07-31', '2003-10-31',
               '2004-01-31', '2004-04-30', '2004-07-31', '2004-10-31',
               '2005-01-31', '2005-04-30', '2005-07-31', '2005-10-31',
               '2006-01-31', '2006-04-30', '2006-07-31', '2006-10-31',
               '2007-01-31', '2007-04-30', '2007-07-31', '2007-10-31',
               '2008-01-31', '2008-04-30', '2008-07-31', '2008-10-31',
               '2009-01-31', '2009-04-30', '2009-07-31', '2009-10-31',
               '2010-01-31', '2010-04-30', '2010-07-31', '2010-10-31',
               '2011-01-31', '2011-04-30', '2011-07-31', '2011-10-31',
               '2012-01-31', '2012-04-30', '2012-07-31', '2012-10-31',
               '2013-01-31', '2013-04-30', '2013-07-31', '2013-10-31',
      

In [3]:
def withinten(y_true, y_pred):
    sum1 = 0
    for i in range(len(y_pred)):
        if(y_pred[i] < 1.1*y_true[i] and y_pred[i] > 0.9*y_true[i]):
            sum1+=1
    return sum1/len(y_pred)

In [4]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y')
seq_train = pd.read_csv('../data/1_before2016.csv', parse_dates=['datadate'],
                                 index_col='datadate', date_parser=dateparse).fillna(0)
seq_test = pd.read_csv('../data/1_after2016.csv', parse_dates=['datadate'],
                                 index_col='datadate', date_parser=dateparse).fillna(0)

r2_list = []
mse_list = []
withinten_list = []
for name, content in seq_train.groupby('tic'):
    if(len(content) < 40):
        continue
    try:
        print("------------------"+name+"-----------------")
        dta = content['actual']
        #dta = seq_train.loc[seq_train['tic']=='ALXN']['actual']
        date_range = pd.date_range(start = dta.index[0], end = dta.index[-1], freq='3M')
        dta = dta.reindex(date_range).fillna(method='ffill')
        dta_test = seq_test.loc[seq_test['tic']==name]['actual']

        decomposition = seasonal_decompose(dta, model='additive', period=4)
        trend = decomposition.trend
        seasonal = decomposition.seasonal
        residual = decomposition.resid

        trend = trend.fillna(method='ffill')
        seasonal = seasonal.fillna(method='ffill')
        residual = residual.fillna(method='ffill')

        trend_evaluate = sm.tsa.arma_order_select_ic(trend, ic=['aic', 'bic'], trend='c', max_ar=4,
                                                max_ma=4)
        #p = trend_evaluate.bic_min_order[0]
        #q = trend_evaluate.bic_min_order[1]
        p = 6
        q = 6
        trend_model = sm.tsa.arima.ARIMA(trend, order=(p, 0, q))
        trendres = trend_model.fit()
        trend_predict_seq = trendres.predict(dta_test.index[0], dta_test.index[-1], dynamic=True)

        residual_evaluate = sm.tsa.arma_order_select_ic(residual, ic=['aic', 'bic'], trend='c', max_ar=4,
                                                max_ma=4)
        p = residual_evaluate.bic_min_order[0]
        q = residual_evaluate.bic_min_order[1]
        residual_model = sm.tsa.arima.ARIMA(residual, order=(p, 0, q))
        residualres = residual_model.fit()
        residual_predict_seq = residualres.predict(dta_test.index[0], dta_test.index[-1], dynamic=True)

        seasonal_predict_seq = seasonal[(dta_test.index[0]-relativedelta(years=6)):(dta_test.index[-1]-relativedelta(years=6))]

        #print(trend_predict_seq, seasonal_predict_seq)
        predict_dates = trend_predict_seq.index
        seasonal_predict_seq.index = predict_dates

        predict_seq = pd.Series(seasonal_predict_seq, index=seasonal_predict_seq.index)
        predict_seq = predict_seq.add(trend_predict_seq)
        predict_seq = predict_seq.add(residual_predict_seq)

        mse = mean_squared_error(dta_test, predict_seq)
        r2 = r2_score(dta_test, predict_seq)
        within10 = withinten(dta_test, predict_seq)

        mse_list.append(mse)
        r2_list.append(r2)
        withinten_list.append(within10)
        break
    except:
        print(name+' not succeed.')
# fig = plt.figure(figsize=(12, 8))
# ax1 = fig.add_subplot(411)
# fig = sm.graphics.tsa.plot_acf(trend, lags=20, ax=ax1)
# ax2 = fig.add_subplot(412)
# fig = sm.graphics.tsa.plot_pacf(trend, lags=20, ax=ax2)
# ax3 = fig.add_subplot(413)
# fig = sm.graphics.tsa.plot_acf(residual, lags=20, ax=ax3)
# ax4 = fig.add_subplot(414)
# fig = sm.graphics.tsa.plot_pacf(residual, lags=20, ax=ax4)

# plt.figure(figsize=(16, 12))
# plt.subplot(411)
# plt.plot(dta, label='Original')
# plt.legend(loc='best')
# plt.subplot(412)
# plt.plot(trend, label='Trend')
# plt.legend(loc='best')
# plt.subplot(413)
# plt.plot(seasonal, label='Seasonarity')
# plt.legend(loc='best')
# plt.subplot(414)
# plt.plot(residual, label='Residual')
# plt.legend(loc='best')
# plt.show()

------------------A-----------------
------------------AAP-----------------
------------------AAPL-----------------
------------------ABC-----------------
------------------ABMD-----------------
------------------ABT-----------------
------------------ACN-----------------
------------------ADBE-----------------
------------------ADI-----------------
------------------ADM-----------------
------------------ADP-----------------
------------------ADSK-----------------
------------------AEE-----------------
------------------AEP-----------------
------------------AES-----------------
------------------AJG-----------------
------------------AKAM-----------------
------------------ALB-----------------
------------------ALGN-----------------
------------------ALK-----------------
------------------ALXN-----------------
ALXN not succeed.
------------------AMAT-----------------
------------------AMD-----------------
------------------AME-----------------
------------------AMGN-----------------


In [8]:
print(np.mean(mse_list), np.mean(r2_list), np.mean(withinten_list))
print(np.min(r2_list), np.max(r2_list), np.std(r2_list))

0.9431179745902453 -2.223222581901191 0.15813899456377759
-28.512867360974223 0.9274601567334434 3.4692648968668145
